### Creating our own simple Data
    1.It will have 3 columns - Age, Gender, Genre
    2.men - (20-25) - HipHop, (26-30) - Jazz, (30-x) - Classical
    3.women - (20-25) - Dance, (26-30) - Accoustic, (30-x) - Classical

In [38]:
import pandas as pd

In [39]:
#Creating a list and then filling data in it.
import random
dataset_size = 100
age = [random.randint(20,50) for i in range(dataset_size)]    

In [40]:
gen = ['Male', 'Female']
gender = [random.choice(gen) for i in range(dataset_size)]

#### Now using the rule described above creating a list for Genre.

In [41]:
def choose_genre(series):
    age_x = series['Age']
    gen_x = series['Gender']
    if age_x >=20 and age_x<=25:
        if gen_x == "Male":
            return "HipHop"
        else:
            return "Dance"
    elif age_x>=26 and age_x<=30:
        if gen_x == "Male":
            return "Jazz"
        else:
            return "Accoustic"
    else:
        return "Classical"

In [42]:
train_df = pd.DataFrame({"Age":age, "Gender":gender})

In [43]:
train_df['Genre'] = train_df.apply(choose_genre, axis='columns')

In [28]:
train_df.head()

,Age,Gender,Genre
0,41,Male,Classical
1,26,Male,Jazz
2,23,Male,HipHop
3,47,Female,Classical
4,30,Female,Accoustic


### Dummy dataset creation complete - as we can see it has age, w.r.t gender and genre.
- We can save this df into csv file for later use, and now we dont need to load it into df as its already loaded

In [44]:
## saving dataframe into csv for later use
train_df.to_csv("music_data.csv")

### Feature Engineering - categorical to numerical, checking nan values - imputing, finding patterns.

In [47]:
## No point of checking nan values, as we created the dataset. But checking for practice.
# train_df.isna().sum()
from sklearn import preprocessing

## Dont want to change the label name, only changing the gender
for col in train_df.columns[:-1]:
    if train_df[col].dtypes == 'object':
        lbl_enc = preprocessing.LabelEncoder()
        lbl_enc_fit = lbl_enc.fit(train_df[col].values)
        train_df[col] = lbl_enc_fit.transform(list(train_df[col].values))

In [50]:
train_df.head()

,Age,Gender,Genre
0,41,0,Classical
1,34,1,Classical
2,42,1,Classical
3,23,0,Dance
4,24,0,Dance


### Creating a input and output data.
- Input data is Age and Gender
- ouptut data(label) is Genre

In [60]:
X = train_df.iloc[:,:-1]
y = train_df.iloc[:,-1]

In [61]:
X.head()

,Age,Gender
0,41,0
1,34,1
2,42,1
3,23,0
4,24,0


In [63]:
y.head()

0    Classical
1    Classical
2    Classical
3        Dance
4        Dance
Name: Genre, dtype: object

### Using Decision tree classifier

In [66]:
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier()
model.fit(X,y)

DecisionTreeClassifier()

### Making prediction

In [68]:
prediction = model.predict([[21, 1], [22,  0]])
print(prediction)

['HipHop' 'Dance']


### We can see, it is making accurate predictions. But still we have not measure accuracy. For that we can split our dataset into train and test, and check accuracy on test dataset.  

In [69]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, shuffle=1)

### Now we can repeat all the steps we did for single dataset.

In [71]:
model_train = DecisionTreeClassifier()
model_train.fit(X_train,y_train)

DecisionTreeClassifier()

In [73]:
## Making predictions
predictions = model_train.predict(X_test)

In [75]:
from sklearn.metrics import accuracy_score
acc = accuracy_score(y_test, predictions)

In [78]:
print(f'Accuracy is - {acc*100}')

Accuracy is - 100.0


#### It is giving 100% accuracy, as our problem statement and dataset is simple enough to give 100% accuracy. If see change in accuracy, we can experiment with reducing train data size or running decision tree again and again.

#### Saving a model

In [81]:
import joblib
joblib.dump(model_train, 'recommender_system.joblib')

['recommender_system.joblib']

### Loading and predicting a datapoint

In [82]:
trained_model = joblib.load('recommender_system.joblib')
trained_model.predict([[22,0]])

array(['Dance'], dtype=object)

### Visualizing our decision tree

In [86]:
from sklearn import tree
tree.export_graphviz(trained_model, 
                     out_file="music-recommend.dot", 
                     feature_names=['Age', 'Gender'], 
                     class_names=sorted(y.unique()),
                     label = 'all',
                     rounded = True, 
                     filled = True)